In [1]:
!pwd

/home/giakhang/dev/pose_sandbox


In [2]:
import os
import sys
import glob

sys.path.append("Hand_pose_estimation_3D/arm_and_hand")
sys.path.append("Hand_pose_estimation_3D")

from dataloader_hand_only_ann_with_anchors import HandLandmarksDataset_ANN_With_Anchors

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from datetime import datetime
import pandas as pd

from utilities import fuse_landmarks_from_two_cameras, convert_to_left_shoulder_coord
from functools import partial
from ann import ANN

import open3d as o3d
from utilities import convert_to_left_shoulder_coord
import time

from csv_writer import columns_to_normalize, fusion_csv_columns_name
from landmarks_scaler import LandmarksScaler

from angle_calculator import LeftArmAngleCalculator
from angle_calculator import LeftHandAngleCalculator

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
arm_hand_fused_names = ["left shoulder", "left elbow", "left hip", "right shoulder",
    "right hip", "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", 
    "THUMB_TIP", "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP",
    "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP",
    "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP",
    "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP", "right elbow",
    "RIGHT_WRIST", "RIGHT_THUMB_CMC", "RIGHT_THUMB_MCP", "RIGHT_THUMB_IP", "RIGHT_THUMB_TIP",
    "RIGHT_INDEX_FINGER_MCP", "RIGHT_INDEX_FINGER_PIP", "RIGHT_INDEX_FINGER_DIP",
    "RIGHT_INDEX_FINGER_TIP", "RIGHT_MIDDLE_FINGER_MCP", "RIGHT_MIDDLE_FINGER_PIP",
    "RIGHT_MIDDLE_FINGER_DIP", "RIGHT_MIDDLE_FINGER_TIP", "RIGHT_RING_FINGER_MCP",
    "RIGHT_RING_FINGER_PIP", "RIGHT_RING_FINGER_DIP", "RIGHT_RING_FINGER_TIP",
    "RIGHT_PINKY_MCP", "RIGHT_PINKY_PIP", "RIGHT_PINKY_DIP", "RIGHT_PINKY_TIP"]

In [4]:
left_arm_angle_calculator = LeftArmAngleCalculator(
    num_chain=3, 
    landmark_dictionary=arm_hand_fused_names)
left_hand_angle_calculator = LeftHandAngleCalculator(
    num_chain=2, 
    landmark_dictionary=arm_hand_fused_names)

In [5]:
num_hand_lmks = 21
num_hand_anchors = 3

DATA_DIR = "data"  
SELECTED_DATE = "*"

MODEL_DATE = "20241107"
MODEL_TIME = "1810"
HIDDEN_DIM_CONTAINER = [num_hand_lmks * 3 * 2, 100, 100, 100, 70, 70]
NUM_HIDDEN_LAYERS = 5
MODEL_NAME = "ann_left_hand"

In [6]:
train_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "train")))
val_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "val")))
test_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "test")))
body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]
train_body_distance_thres = 550
train_leftarm_distance_thres = 550
train_lefthand_distance_thres = 200
val_body_distance_thres=450,
val_leftarm_distance_thres=450,
val_lefthand_distance_thres=150,

input_scaler_path = "Hand_pose_estimation_3D/arm_and_hand/runs/{}/{}/{}-{}/input_scaler.pkl".format(MODEL_NAME, MODEL_DATE, MODEL_DATE, MODEL_TIME)
output_scaler_path = "Hand_pose_estimation_3D/arm_and_hand/runs/{}/{}/{}-{}/output_scaler.pkl".format(MODEL_NAME, MODEL_DATE, MODEL_DATE, MODEL_TIME)
input_scaler = LandmarksScaler(scaler_path=input_scaler_path)
output_scaler = LandmarksScaler(scaler_path=output_scaler_path)

train_dataset = HandLandmarksDataset_ANN_With_Anchors(train_paths, 
    arm_hand_fused_names,
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True,
    use_thumb_as_anchor=True,
    input_scaler=input_scaler,
    output_scaler=output_scaler)
val_dataset = HandLandmarksDataset_ANN_With_Anchors(val_paths,
    arm_hand_fused_names,
    body_lines,
    lefthand_lines,
    val_body_distance_thres,
    val_leftarm_distance_thres,
    val_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True,
    use_thumb_as_anchor=True,
    input_scaler=input_scaler,
    output_scaler=output_scaler)
test_dataset = HandLandmarksDataset_ANN_With_Anchors(test_paths,
    arm_hand_fused_names,
    body_lines,
    lefthand_lines,
    val_body_distance_thres,
    val_leftarm_distance_thres,
    val_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True,
    use_thumb_as_anchor=True,
    input_scaler=input_scaler,
    output_scaler=output_scaler)

In [7]:
num_hand_lmks = 21
left_wrist_idx = 5

In [8]:
INPUT_DIM = (num_hand_lmks * 3 * 2) + (5 * 3)  + (num_hand_anchors * 3)
OUTPUT_DIM = (num_hand_lmks - num_hand_anchors) * 3
HIDDEN_DIM_CONTAINER = [num_hand_lmks * 3 * 2, 100, 100, 100, 70, 70]
NUM_HIDDEN_LAYERS = 5
DROPOUT_RATE = 0.1

MODEL_NAME = "ann_left_hand"
model_weight_path = "Hand_pose_estimation_3D/arm_and_hand/runs/{}/{}/{}-{}/{}_{}_layers_best.pth".format(
    MODEL_NAME, MODEL_DATE, MODEL_DATE, MODEL_TIME, MODEL_NAME, NUM_HIDDEN_LAYERS)

ann_model = ANN(
    input_dim=INPUT_DIM,
    output_dim=OUTPUT_DIM,
    hidden_dim_container=HIDDEN_DIM_CONTAINER,
    num_hidden_layers=NUM_HIDDEN_LAYERS,
    dropout_rate=DROPOUT_RATE
)

ann_model.load_state_dict(torch.load(model_weight_path))
ann_model.to("cuda")
ann_model.eval()

ANN(
  (network): Sequential(
    (0): Linear(in_features=150, out_features=126, bias=True)
    (1): BatchNorm1d(126, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): SiLU()
    (3): Linear(in_features=126, out_features=100, bias=True)
    (4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (5): SiLU()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (8): SiLU()
    (9): Linear(in_features=100, out_features=100, bias=True)
    (10): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (11): SiLU()
    (12): Linear(in_features=100, out_features=70, bias=True)
    (13): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (14): SiLU()
    (15): Linear(in_features=70, out_features=70, bias=True)
    (16): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=Fal

In [9]:
train_dataset._outputs.shape

(23902, 54)

In [10]:
input_data = train_dataset._inputs
input_row = torch.Tensor(input_data)
input_row = input_row.to("cuda")

pred_output = ann_model(input_row)
pred_output.shape

torch.Size([23902, 54])

In [11]:
pred_output = pred_output.detach().to("cpu").numpy()

In [12]:
body = train_dataset._body_outputs
pred_train_left_hand = output_scaler.minmax_scaler.inverse_transform(pred_output)
hand_anchors = train_dataset._hand_anchors

In [13]:
pred_train_left_hand = pred_train_left_hand.reshape(pred_train_left_hand.shape[0], 3, -1)

In [14]:
for i, joint_idx in enumerate(train_dataset.hand_anchor_indexes):
    pred_train_left_hand = np.insert(
        pred_train_left_hand,
        [joint_idx],
        hand_anchors[..., i][..., None],
        axis=-1
    )

In [15]:
pred_target = np.concatenate([body, pred_train_left_hand], axis=-1)

In [16]:
pred_target = np.transpose(pred_target, (0, 2, 1))

In [17]:
gt_train_left_hand = output_scaler.minmax_scaler.inverse_transform(train_dataset._outputs)
gt_train_left_hand = gt_train_left_hand.reshape(gt_train_left_hand.shape[0], 3, -1)

for i, joint_idx in enumerate(train_dataset.hand_anchor_indexes):
    gt_train_left_hand = np.insert(
        gt_train_left_hand,
        [joint_idx],
        hand_anchors[..., i][..., None],
        axis=-1
    )


gt_target = np.concatenate([body, gt_train_left_hand], axis=-1)
gt_target = np.transpose(gt_target, (0, 2, 1))

Calculate angles' errors

In [18]:
#pred_target[:, left_wrist_idx, :] = gt_target[:, left_wrist_idx, :]

In [19]:
pred_target.shape

(23902, 26, 3)

In [20]:
pred_target.shape

(23902, 26, 3)

In [21]:
gt_target.shape

(23902, 26, 3)

In [22]:
pred_angles = []

for i in range(pred_target.shape[0]):
    output = pred_target[i]
    
    train_pred_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        output,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(train_pred_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(train_pred_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    pred_angles.append(angles)

pred_angles = np.array(pred_angles)

In [23]:
gt_angles = []

for i in range(gt_target.shape[0]):
    gt = gt_target[i]
    train_gt_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        gt,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(train_gt_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(train_gt_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    gt_angles.append(angles)

gt_angles = np.array(gt_angles)

In [24]:
angles_names = ["left_arm_joint_01",
                "left_arm_joint_02",
                "left_arm_joint_03",
                "left_arm_joint_04",
                "left_arm_joint_05",
                "left_arm_joint_06",
                "left_THUMB_joint_01",
                "left_THUMB_joint_02",
                "left_THUMB_joint_03",
                "left_INDEX_joint_01",
                "left_INDEX_joint_02",
                "left_INDEX_joint_03",
                "left_MIDDLE_joint_01",
                "left_MIDDLE_joint_02",
                "left_MIDDLE_joint_03",
                "left_RING_joint_01",
                "left_RING_joint_02",
                "left_RING_joint_03",
                "left_PINKY_joint_01",
                "left_PINKY_joint_02",
                "left_PINKY_joint_03"]

In [25]:
angles_errors = np.mean(np.abs(gt_angles - pred_angles), axis=0)

In [26]:
assert len(angles_errors) == len(angles_names)

for joint_error, joint_name in zip(angles_errors, angles_names):
    print(f"{joint_name} mean error: {round(joint_error, 2)}")

left_arm_joint_01 mean error: 0.0
left_arm_joint_02 mean error: 0.0
left_arm_joint_03 mean error: 0.0
left_arm_joint_04 mean error: 0.0
left_arm_joint_05 mean error: 0.0
left_arm_joint_06 mean error: 0.0
left_THUMB_joint_01 mean error: 8.96
left_THUMB_joint_02 mean error: 13.29
left_THUMB_joint_03 mean error: 8.26
left_INDEX_joint_01 mean error: 12.31
left_INDEX_joint_02 mean error: 17.73
left_INDEX_joint_03 mean error: 24.52
left_MIDDLE_joint_01 mean error: 9.91
left_MIDDLE_joint_02 mean error: 15.44
left_MIDDLE_joint_03 mean error: 23.53
left_RING_joint_01 mean error: 8.32
left_RING_joint_02 mean error: 12.66
left_RING_joint_03 mean error: 16.01
left_PINKY_joint_01 mean error: 8.67
left_PINKY_joint_02 mean error: 13.28
left_PINKY_joint_03 mean error: 14.91


In [ ]:
target_output = pred_target
time_sleep = .5
x = np.array([[500, 0, 0],
    [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)
lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)

bounding_box = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(np.array([[500,0 ,0], [0, 0, 0]])),
    lines=o3d.utility.Vector2iVector([[0, 0]])
)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)
vis.add_geometry(bounding_box)

for i in range(target_output.shape[0]):
    output = target_output[i, ...]  # shape: (26, 3)

    lines = body_lines.copy()
    lines.extend(lefthand_lines)

    points, _ = convert_to_left_shoulder_coord(output,
        arm_hand_fused_names)

    pcd.points = o3d.utility.Vector3dVector(points)
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points)  
    line_set.lines = o3d.utility.Vector2iVector(lines) 
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # Draw cuboid
    min_x, min_y, min_z = np.min(points, axis=0)
    max_x, max_y, max_z = np.max(points, axis=0)
    vertices = [
        [min_x, min_y, min_z], [min_x, min_y, max_z], [min_x, max_y, min_z], [min_x, max_y, max_z],
        [max_x, min_y, min_z], [max_x, min_y, max_z], [max_x, max_y, min_z], [max_x, max_y, max_z]
    ]
    vertices = np.array(vertices) 
    edges = [
        [0, 1], [1, 3], [3, 2], [2, 0],  
        [4, 5], [5, 7], [7, 6], [6, 4],  
        [0, 4], [1, 5], [2, 6], [3, 7]   
    ]
    bounding_box.points = o3d.utility.Vector3dVector(vertices)
    bounding_box.lines = o3d.utility.Vector2iVector(edges)
    bbox_colors = [[0, 1, 0] for _ in range(len(edges))]  
    bounding_box.colors = o3d.utility.Vector3dVector(bbox_colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.update_geometry(bounding_box)
    vis.poll_events()
    vis.update_renderer()
        
    time.sleep(time_sleep)

vis.destroy_window()

KeyboardInterrupt: 

In [ ]:
target_output = gt_target
time_sleep = .001
x = np.array([[500, 0, 0],
    [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)
lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)

bounding_box = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(np.array([[500,0 ,0], [0, 0, 0]])),
    lines=o3d.utility.Vector2iVector([[0, 0]])
)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)
vis.add_geometry(bounding_box)

for i in range(target_output.shape[0]):
    output = target_output[i, ...]  # shape: (26, 3)

    lines = body_lines.copy()
    lines.extend(lefthand_lines)

    points, _ = convert_to_left_shoulder_coord(output,
        arm_hand_fused_names)

    pcd.points = o3d.utility.Vector3dVector(points)
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points)  
    line_set.lines = o3d.utility.Vector2iVector(lines) 
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # Draw cuboid
    min_x, min_y, min_z = np.min(points, axis=0)
    max_x, max_y, max_z = np.max(points, axis=0)
    vertices = [
        [min_x, min_y, min_z], [min_x, min_y, max_z], [min_x, max_y, min_z], [min_x, max_y, max_z],
        [max_x, min_y, min_z], [max_x, min_y, max_z], [max_x, max_y, min_z], [max_x, max_y, max_z]
    ]
    vertices = np.array(vertices) 
    edges = [
        [0, 1], [1, 3], [3, 2], [2, 0],  
        [4, 5], [5, 7], [7, 6], [6, 4],  
        [0, 4], [1, 5], [2, 6], [3, 7]   
    ]
    bounding_box.points = o3d.utility.Vector3dVector(vertices)
    bounding_box.lines = o3d.utility.Vector2iVector(edges)
    bbox_colors = [[0, 1, 0] for _ in range(len(edges))]  
    bounding_box.colors = o3d.utility.Vector3dVector(bbox_colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.update_geometry(bounding_box)
    vis.poll_events()
    vis.update_renderer()
        
    time.sleep(time_sleep)

vis.destroy_window()

KeyboardInterrupt: 

# Inspect val set

In [30]:
val_dataset._outputs.shape

(2379, 54)

In [31]:
val_input_data = val_dataset._inputs
val_input_data = torch.Tensor(val_input_data)
val_input_data = val_input_data.to("cuda")

val_pred = ann_model(val_input_data)
val_pred.shape

torch.Size([2379, 54])

In [32]:
val_pred = val_pred.detach().to("cpu").numpy()

In [33]:
body = val_dataset._body_outputs
pred_val_left_hand = output_scaler.minmax_scaler.inverse_transform(val_pred)
hand_anchors = val_dataset._hand_anchors

In [34]:
pred_val_left_hand = pred_val_left_hand.reshape(pred_val_left_hand.shape[0], 3, -1)

In [35]:
for i, joint_idx in enumerate(val_dataset.hand_anchor_indexes):
    pred_val_left_hand = np.insert(
        pred_val_left_hand,
        [joint_idx],
        hand_anchors[..., i][..., None],
        axis=-1
    )

In [36]:
val_pred_target = np.concatenate([body, pred_val_left_hand], axis=-1)

In [37]:
val_pred_target = np.transpose(val_pred_target, (0, 2, 1))

In [39]:
val_gt_left_hand = output_scaler.minmax_scaler.inverse_transform(val_dataset._outputs)
val_gt_left_hand = val_gt_left_hand.reshape(val_gt_left_hand.shape[0], 3, -1)

for i, joint_idx in enumerate(val_dataset.hand_anchor_indexes):
    val_gt_left_hand = np.insert(
        val_gt_left_hand,
        [joint_idx],
        hand_anchors[..., i][..., None],
        axis=-1
    )

val_gt_target = np.concatenate([body, val_gt_left_hand], axis=-1)
val_gt_target = np.transpose(val_gt_target, (0, 2, 1))

In [40]:
val_pred_target.shape

(2379, 26, 3)

In [41]:
val_gt_target.shape

(2379, 26, 3)

In [42]:
pred_angles = []

for i in range(val_pred_target.shape[0]):
    output = val_pred_target[i]
    
    val_pred_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        output,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(val_pred_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(val_pred_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    pred_angles.append(angles)

pred_angles = np.array(pred_angles)

In [43]:
gt_angles = []

for i in range(val_gt_target.shape[0]):
    gt = val_gt_target[i]
    val_gt_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        gt,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(val_gt_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(val_gt_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    gt_angles.append(angles)

gt_angles = np.array(gt_angles)

In [44]:
angles_names = ["left_arm_joint_01",
                "left_arm_joint_02",
                "left_arm_joint_03",
                "left_arm_joint_04",
                "left_arm_joint_05",
                "left_arm_joint_06",
                "left_THUMB_joint_01",
                "left_THUMB_joint_02",
                "left_THUMB_joint_03",
                "left_INDEX_joint_01",
                "left_INDEX_joint_02",
                "left_INDEX_joint_03",
                "left_MIDDLE_joint_01",
                "left_MIDDLE_joint_02",
                "left_MIDDLE_joint_03",
                "left_RING_joint_01",
                "left_RING_joint_02",
                "left_RING_joint_03",
                "left_PINKY_joint_01",
                "left_PINKY_joint_02",
                "left_PINKY_joint_03"]

In [45]:
angles_errors = np.mean(np.abs(gt_angles - pred_angles), axis=0)

In [46]:
assert len(angles_errors) == len(angles_names)

for joint_error, joint_name in zip(angles_errors, angles_names):
    print(f"{joint_name} mean error: {round(joint_error, 2)}")

left_arm_joint_01 mean error: 0.0
left_arm_joint_02 mean error: 0.0
left_arm_joint_03 mean error: 0.0
left_arm_joint_04 mean error: 0.0
left_arm_joint_05 mean error: 0.0
left_arm_joint_06 mean error: 0.0
left_THUMB_joint_01 mean error: 10.91
left_THUMB_joint_02 mean error: 11.77
left_THUMB_joint_03 mean error: 7.59
left_INDEX_joint_01 mean error: 13.77
left_INDEX_joint_02 mean error: 20.98
left_INDEX_joint_03 mean error: 23.99
left_MIDDLE_joint_01 mean error: 13.14
left_MIDDLE_joint_02 mean error: 18.79
left_MIDDLE_joint_03 mean error: 23.22
left_RING_joint_01 mean error: 9.52
left_RING_joint_02 mean error: 15.51
left_RING_joint_03 mean error: 16.18
left_PINKY_joint_01 mean error: 10.35
left_PINKY_joint_02 mean error: 15.58
left_PINKY_joint_03 mean error: 15.13


In [ ]:
target_output = val_pred_target
time_sleep = .1
x = np.array([[500, 0, 0],
    [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)
lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)

bounding_box = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(np.array([[500,0 ,0], [0, 0, 0]])),
    lines=o3d.utility.Vector2iVector([[0, 0]])
)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)
vis.add_geometry(bounding_box)

for i in range(target_output.shape[0]):
    output = target_output[i, ...]  # shape: (26, 3)

    lines = body_lines.copy()
    lines.extend(lefthand_lines)

    points, _ = convert_to_left_shoulder_coord(output,
        arm_hand_fused_names)

    pcd.points = o3d.utility.Vector3dVector(points)
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points)  
    line_set.lines = o3d.utility.Vector2iVector(lines) 
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # Draw cuboid
    min_x, min_y, min_z = np.min(points, axis=0)
    max_x, max_y, max_z = np.max(points, axis=0)
    vertices = [
        [min_x, min_y, min_z], [min_x, min_y, max_z], [min_x, max_y, min_z], [min_x, max_y, max_z],
        [max_x, min_y, min_z], [max_x, min_y, max_z], [max_x, max_y, min_z], [max_x, max_y, max_z]
    ]
    vertices = np.array(vertices) 
    edges = [
        [0, 1], [1, 3], [3, 2], [2, 0],  
        [4, 5], [5, 7], [7, 6], [6, 4],  
        [0, 4], [1, 5], [2, 6], [3, 7]   
    ]
    bounding_box.points = o3d.utility.Vector3dVector(vertices)
    bounding_box.lines = o3d.utility.Vector2iVector(edges)
    bbox_colors = [[0, 1, 0] for _ in range(len(edges))]  
    bounding_box.colors = o3d.utility.Vector3dVector(bbox_colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.update_geometry(bounding_box)
    vis.poll_events()
    vis.update_renderer()
        
    time.sleep(time_sleep)

vis.destroy_window()

KeyboardInterrupt: 

# Inspect test set

In [ ]:
test_input = test_dataset._inputs
test_input.shape

(1114, 288)

In [ ]:
test_gt = test_dataset._outputs
test_gt = test_gt.reshape(-1, 3, 48)
test_gt = np.transpose(test_gt, (0, 2, 1))
test_gt.shape

(1114, 48, 3)

In [ ]:
test_input = torch.Tensor(test_input)
test_input = test_input.to("cuda")
test_preds = ann_model(test_input)
test_preds = test_preds.detach().to("cpu").numpy()
test_preds = test_preds.reshape(-1, 3, 48)
test_preds = np.transpose(test_preds, (0, 2, 1))
test_preds.shape

(1114, 48, 3)

In [ ]:
mae_of_first_26_joints = np.average(np.abs(test_gt[:, :26, :] - test_preds[:, :26, :]))
print("Mean abs. error of the first 26 joints: {}(mm)".format(round(mae_of_first_26_joints, 2)))

Mean abs. error of the first 26 joints: 17.51(mm)


In [ ]:
test_preds = test_preds[:, :26, :]
print(test_preds.shape)

(1114, 26, 3)


In [ ]:
test_gt = test_gt[:, :26, :]
print(test_gt.shape)

(1114, 26, 3)


In [ ]:
pred_angles = []

for i in range(test_preds.shape[0]):
    test_pred = test_preds[i]
    test_pred_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        test_pred,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(test_pred_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(test_pred_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    pred_angles.append(angles)

pred_angles = np.array(pred_angles)

In [ ]:
gt_angles = []

for i in range(test_gt.shape[0]):
    gt = test_gt[i]
    test_gt_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        gt,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(test_gt_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(test_gt_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    gt_angles.append(angles)

gt_angles = np.array(gt_angles)

In [ ]:
angles_errors = np.mean(np.abs(gt_angles - pred_angles), axis=0)

In [ ]:
angles_names = ["left_arm_joint_01",
                "left_arm_joint_02",
                "left_arm_joint_03",
                "left_arm_joint_04",
                "left_arm_joint_05",
                "left_arm_joint_06",
                "left_THUMB_joint_01",
                "left_THUMB_joint_02",
                "left_THUMB_joint_03",
                "left_INDEX_joint_01",
                "left_INDEX_joint_02",
                "left_INDEX_joint_03",
                "left_MIDDLE_joint_01",
                "left_MIDDLE_joint_02",
                "left_MIDDLE_joint_03",
                "left_RING_joint_01",
                "left_RING_joint_02",
                "left_RING_joint_03",
                "left_PINKY_joint_01",
                "left_PINKY_joint_02",
                "left_PINKY_joint_03"]

In [ ]:
assert len(angles_names) == len(angles_errors)

for joint_error, joint_name in zip(angles_errors, angles_names):
    print(f"{joint_name} mean error: {round(joint_error, 2)}")

left_arm_joint_01 mean error: 7.88
left_arm_joint_02 mean error: 8.21
left_arm_joint_03 mean error: 22.49
left_arm_joint_04 mean error: 9.95
left_arm_joint_05 mean error: 47.9
left_arm_joint_06 mean error: 13.42
left_THUMB_joint_01 mean error: 14.81
left_THUMB_joint_02 mean error: 17.03
left_THUMB_joint_03 mean error: 6.85
left_INDEX_joint_01 mean error: 14.69
left_INDEX_joint_02 mean error: 17.73
left_INDEX_joint_03 mean error: 22.85
left_MIDDLE_joint_01 mean error: 16.8
left_MIDDLE_joint_02 mean error: 22.12
left_MIDDLE_joint_03 mean error: 27.35
left_RING_joint_01 mean error: 18.25
left_RING_joint_02 mean error: 23.08
left_RING_joint_03 mean error: 28.47
left_PINKY_joint_01 mean error: 19.52
left_PINKY_joint_02 mean error: 27.18
left_PINKY_joint_03 mean error: 25.69


In [ ]:
target_output = test_preds # shape: (N, 144)
time_sleep = .1
x = np.array([[500, 0, 0],
    [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)
lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)

bounding_box = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(np.array([[500,0 ,0], [0, 0, 0]])),
    lines=o3d.utility.Vector2iVector([[0, 0]])
)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)
vis.add_geometry(bounding_box)

for i in range(target_output.shape[0]):
    output = target_output[i, ...]  # shape: (144)
    output = output.reshape(3, 48)  # shape: (3, 48)
    output = output.T  # shape: (48, 3)
    output = output[:26, ...]  # shape: (26, 3)

    lines = body_lines.copy()
    lines.extend(lefthand_lines)

    points, _ = convert_to_left_shoulder_coord(output,
        arm_hand_fused_names)

    pcd.points = o3d.utility.Vector3dVector(points)
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points)  
    line_set.lines = o3d.utility.Vector2iVector(lines) 
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # Draw cuboid
    min_x, min_y, min_z = np.min(points, axis=0)
    max_x, max_y, max_z = np.max(points, axis=0)
    vertices = [
        [min_x, min_y, min_z], [min_x, min_y, max_z], [min_x, max_y, min_z], [min_x, max_y, max_z],
        [max_x, min_y, min_z], [max_x, min_y, max_z], [max_x, max_y, min_z], [max_x, max_y, max_z]
    ]
    vertices = np.array(vertices) 
    edges = [
        [0, 1], [1, 3], [3, 2], [2, 0],  
        [4, 5], [5, 7], [7, 6], [6, 4],  
        [0, 4], [1, 5], [2, 6], [3, 7]   
    ]
    bounding_box.points = o3d.utility.Vector3dVector(vertices)
    bounding_box.lines = o3d.utility.Vector2iVector(edges)
    bbox_colors = [[0, 1, 0] for _ in range(len(edges))]  
    bounding_box.colors = o3d.utility.Vector3dVector(bbox_colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.update_geometry(bounding_box)
    vis.poll_events()
    vis.update_renderer()
        
    time.sleep(time_sleep)

vis.destroy_window()